In [4]:
import numpy as np
import pandas as pd
#import nltk
import string
import re

# import 5 emotions training csv

In [5]:
df = pd.read_csv(r'C:\Users\Shibbs\Desktop\Projects\Sentiment analysis\My dataset\5emotions_final.csv')
df

,Unnamed: 0,index,labels,tweet
0,0,2,neutral,we want to trade with someone who has houston...
1,1,3,worry,repinging why didnt you go to prom bc my bf d...
2,2,5,worry,hmmm httpwwwdjherocom is down
3,3,6,sadness,charlene my love i miss you
4,4,7,sadness,im sorry at least its friday
...,...,...,...,...
19950,19950,31250,happiness,succesfully following tayla
19951,19951,31251,love,happy mothers day all my love
19952,19952,31252,love,happy mothers day to all the mommies out there...
19953,19953,31253,happiness,wassup beautiful follow me peep out my new h...


In [6]:
df = df.drop(columns = ['index', 'Unnamed: 0'])

In [7]:
df

,labels,tweet
0,neutral,we want to trade with someone who has houston...
1,worry,repinging why didnt you go to prom bc my bf d...
2,worry,hmmm httpwwwdjherocom is down
3,sadness,charlene my love i miss you
4,sadness,im sorry at least its friday
...,...,...
19950,happiness,succesfully following tayla
19951,love,happy mothers day all my love
19952,love,happy mothers day to all the mommies out there...
19953,happiness,wassup beautiful follow me peep out my new h...


In [18]:
df['tweet'] = df['tweet'].str.replace('http\S+|www.\S+|pic.\S+|@\S+|#\S+', '', case=False)

In [19]:
df

,labels,tweet
0,neutral,we want to trade with someone who has houston...
1,worry,repinging why didnt you go to prom bc my bf d...
2,worry,hmmm is down
3,sadness,charlene my love i miss you
4,sadness,im sorry at least its friday
...,...,...
19950,happiness,succesfully following tayla
19951,love,happy mothers day all my love
19952,love,happy mothers day to all the mommies out there...
19953,happiness,wassup beautiful follow me peep out my new h...


In [20]:
df['labels'].value_counts()

neutral      4088
worry        4056
happiness    4008
sadness      3962
love         3841
Name: labels, dtype: int64

In [11]:
'''neutral_list = []

for index, row in df.iterrows():
    if df['labels'][index] == 'sadness':
        neutral_list.append(index)'''
    
    

"neutral_list = []\n\nfor index, row in df.iterrows():\n    if df['labels'][index] == 'sadness':\n        neutral_list.append(index)"

In [28]:
'''import random

random.shuffle(neutral_list)'''

In [29]:
'''df = df.drop(neutral_list[:1200])'''

In [12]:
list = []

for index, row in df.iterrows():
    desc = row['tweet']
    list.append(str(desc))

In [21]:
list = []
for index, row in df.iterrows():
    desc = row['tweet']
    list.append(" ".join(re.findall("[a-zA-Z]+", str(desc))))

In [22]:
list

['we want to trade with someone who has houston tickets but no one will',
 'repinging why didnt you go to prom bc my bf didnt like my friends',
 'hmmm is down',
 'charlene my love i miss you',
 'im sorry at least its friday',
 'choked on her retainers',
 'ugh i have to beat this stupid song to get to the next rude',
 'agreed',
 'so sleepy again and its not even that late i fail once again',
 'lady gaga tweeted about not being impressed by her video leaking just so you know',
 'how are you convinced that i have always wanted you what signals did i give off i think i just lost another friend',
 'no to talks at the balisage markup conference program online at via to',
 'i ate something i dont know what it is why do i keep telling things about food',
 'so tired and i think im definitely going to get an ear infection going to bed early for once',
 'im sorry people are so rude to you isaac they should get some manners and know better than to be so lewd',
 'damm servers still down i need to h

In [16]:
df.to_csv('5emotions_final_processed.csv')

In [23]:
maxLength = 57

In [24]:
print(maxLength )

57


In [25]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list)
x = tokenizer.texts_to_sequences(list)
x = pad_sequences(x, maxlen=maxLength, padding='pre', truncating='post')

In [26]:
y = pd.get_dummies(df['labels']).values

In [27]:
y[10]

array([0, 0, 0, 1, 0], dtype=uint8)

In [28]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.20)

In [29]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import LSTM, Bidirectional
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import SpatialDropout1D, Dropout
import keras
#from keras.layers import Conv1D, MaxPooling1D
embedding_input = len(tokenizer.word_index) + 1
model = Sequential()
model.add(Embedding(embedding_input, 256, input_length = maxLength))
model.add(SpatialDropout1D(0.25))
model.add(Conv1D(192,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(81,
                 5,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=2))
'''model.add(Conv1D(128,
                 7,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=2))'''
#model.add(Conv1D(512, 3, padding='valid', activation='relu',strides=1))
#model.add(MaxPooling1D(pool_size=2))
#model.add(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
#model.add(Bidirectional(LSTM(64, dropout=0.2)))
#model.add(LSTM(32, dropout = 0.2, return_sequences=True)) # returns a sequence of vectors of dimension 32
#model.add(LSTM(128, return_sequences=True, dropout = 0.7))  # returns a sequence of vectors of dimension 32
#model.add(LSTM(128, dropout = 0.7, recurrent_dropout = 0.7, return_sequences=True))
#model.add(LSTM(64, dropout = 0.5, recurrent_dropout = 0.5, return_sequences=True))
model.add(LSTM(32, dropout = 0.7, recurrent_dropout = 0.7))
#model.add(Dropout(0.5))
#model.add(LSTM(128))
#model.add(Dropout(0.5))
#model.add(Bidirectional(LSTM(64, dropout=0.5)))
#model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
#model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
#model.add(Dense(16,activation='relu'))
#model.add(Dropout(0.8))
model.add(Dense(5, activation='softmax'))
#optimizer = keras.optimizers.Adam(lr=0.0009)
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 57, 256)           5500672   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 57, 256)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 55, 192)           147648    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 27, 192)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 23, 81)            77841     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 11, 81)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)               

In [30]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs = 20,  batch_size = 32)

c:\users\shibbs\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 15964 samples, validate on 3991 samples
Epoch 1/20
15964/15964 [==============================] - 37s 2ms/step - loss: 1.5639 - accuracy: 0.2707 - val_loss: 1.4697 - val_accuracy: 0.3428
Epoch 2/20
15964/15964 [==============================] - 34s 2ms/step - loss: 1.4231 - accuracy: 0.3731 - val_loss: 1.4661 - val_accuracy: 0.3571
Epoch 3/20
15964/15964 [==============================] - 35s 2ms/step - loss: 1.2871 - accuracy: 0.4430 - val_loss: 1.5407 - val_accuracy: 0.3550
Epoch 4/20
15964/15964 [==============================] - 35s 2ms/step - loss: 1.1473 - accuracy: 0.5175 - val_loss: 1.6319 - val_accuracy: 0.3636
Epoch 5/20
15964/15964 [==============================] - 35s 2ms/step - loss: 1.0066 - accuracy: 0.5913 - val_loss: 1.8025 - val_accuracy: 0.3415
Epoch 6/20
15964/15964 [==============================] - 35s 2ms/step - loss: 0.8770 - accuracy: 0.6559 - val_loss: 1.9614 - val_accuracy: 0.3320
Epoch 7/20
15964/15964 [==============================] - 35s 2ms/ste

In [43]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import GRU
from keras.layers import SpatialDropout1D, Dropout
import keras
from keras.layers import Conv1D, MaxPooling1D

embedding_input = len(tokenizer.word_index) + 1

model = Sequential()
model.add(Embedding(embedding_input, 256, input_length = maxLength))
model.add(SpatialDropout1D(0.25))
model.add(Conv1D(256,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(128,
                 5,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64,
                 7,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=2))
#model.add(Conv1D(512, 3, padding='valid', activation='relu',strides=1))
#model.add(MaxPooling1D(pool_size=2))
#model.add(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
#model.add(Bidirectional(LSTM(64, dropout=0.2)))
#model.add(LSTM(32, dropout = 0.2, return_sequences=True)) # returns a sequence of vectors of dimension 32
#model.add(LSTM(128, return_sequences=True, dropout = 0.7))  # returns a sequence of vectors of dimension 32
model.add(GRU(64, dropout = 0.5, recurrent_dropout = 0.5, return_sequences=True))
model.add(GRU(32, dropout = 0.5, recurrent_dropout = 0.5, return_sequences=True))
model.add(GRU(16, dropout = 0.5, recurrent_dropout = 0.5))
#model.add(Bidirectional(LSTM(64, dropout=0.5)))
#model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
#model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
#model.add(Dense(16,activation='relu'))
#model.add(Dropout(0.8))
model.add(Dense(5, activation='softmax'))
#optimizer = keras.optimizers.Adam(lr=0.0007)
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 57, 256)           5764352   
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 57, 256)           0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 55, 256)           196864    
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 27, 256)           0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 23, 128)           163968    
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 11, 128)           0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 5, 64)            

In [44]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs = 20,  batch_size = 32)

Train on 15964 samples, validate on 3991 samples
Epoch 1/20
15964/15964 [==============================] - 39s 2ms/step - loss: 1.6088 - accuracy: 0.2114 - val_loss: 1.5986 - val_accuracy: 0.2440
Epoch 2/20
15964/15964 [==============================] - 35s 2ms/step - loss: 1.5958 - accuracy: 0.2442 - val_loss: 1.5967 - val_accuracy: 0.2483
Epoch 3/20
15964/15964 [==============================] - 35s 2ms/step - loss: 1.5673 - accuracy: 0.2781 - val_loss: 1.6062 - val_accuracy: 0.2543
Epoch 4/20
15964/15964 [==============================] - 35s 2ms/step - loss: 1.5135 - accuracy: 0.3075 - val_loss: 1.6332 - val_accuracy: 0.2603
Epoch 5/20
15964/15964 [==============================] - 35s 2ms/step - loss: 1.4401 - accuracy: 0.3488 - val_loss: 1.6492 - val_accuracy: 0.2749
Epoch 6/20
15964/15964 [==============================] - 35s 2ms/step - loss: 1.3494 - accuracy: 0.3921 - val_loss: 1.7040 - val_accuracy: 0.2826
Epoch 7/20
15964/15964 [==============================] - 35s 2ms/ste

In [51]:
model.save('cnn_lstm_gpu1_7.03.h5')

# import cleaned csv

In [32]:
df0 = pd.read_csv(r'C:\Users\Shibbs\Desktop\Projects\Sentiment analysis\My dataset\mumbai_cleaned_new.csv')

In [33]:
df0

,hashtags,link,tweet
0,"['#coronavirus', '#missionbeginagain', '#unloc...",https://twitter.com/mumbaimatterz/status/12732...,mumbai coronavirus updates missionbeginagain u...
1,"['#covid19india', '#unlock2', '#mumbai', '#unl...",https://twitter.com/yogeshsakh/status/12732559...,seeing the current covid india situation and t...
2,"['#covidー19', '#coronavirus', '#maharashtrafig...",https://twitter.com/Maha_MEDD/status/127320617...,a map prepared by csir neeri mumbai showing co...
3,"['#unlock1', '#mumbai']",https://twitter.com/theprasannata/status/12731...,i think probably us young people in the office...
4,"['#missionbeginagain', '#mumbai', '#unlock1']",https://twitter.com/mumbaiat24am/status/127318...,missionbeginagain mumbai crowds surge on day o...
...,...,...,...
353,"['#covid_19india', '#mumbai', '#socialdistanci...",https://twitter.com/MonilDalal/status/12699036...,government has started the economy but citizen...
354,"['#coronapandemic', '#mumbai', '#mumbaikars', ...",https://twitter.com/NBhalwankar/status/1269896...,another shocker how can the coronapandemic get...
355,"['#mumbai', '#monday', '#mondayblues', '#monda...",https://twitter.com/ompsyram/status/1269896580...,the new normal mumbai monday mondayblues monda...
356,"['#unlock1', '#covid19india']",https://twitter.com/ScribeUpma/status/12698952...,mumbai on work mode unlock covid india


In [34]:
def prediction(dataframe):
    
    list1 = []
    list2 = []
    preds = []

    for index,row in dataframe.iterrows():
        desc = row['tweet']
        list1.append(desc)
        
    for i in list1:
        
        list1 = pad_sequences(tokenizer.texts_to_sequences([i]), truncating='post', maxlen=maxLength)
        score = model.predict([list1])
        preds.append(score)
        emotions = np.array(preds)
        
    for i in range(0, len(emotions)):
        if np.argmax(emotions[i][0]) == 0:
            list2.append('happiness')
        elif np.argmax(emotions[i][0]) == 1:
            list2.append('joy')
        elif np.argmax(emotions[i][0]) == 2:
            list2.append('neutral')
        elif np.argmax(emotions[i][0]) == 3:  
            list2.append('sadness')
        elif np.argmax(emotions[i][0]) == 4:
            list2.append('worry')
        
    
    return list2

In [35]:
sentiments = prediction(df0)

In [36]:
df0['sentiment'] = sentiments

In [37]:
df0

,hashtags,link,tweet,sentiment
0,"['#coronavirus', '#missionbeginagain', '#unloc...",https://twitter.com/mumbaimatterz/status/12732...,mumbai coronavirus updates missionbeginagain u...,worry
1,"['#covid19india', '#unlock2', '#mumbai', '#unl...",https://twitter.com/yogeshsakh/status/12732559...,seeing the current covid india situation and t...,worry
2,"['#covidー19', '#coronavirus', '#maharashtrafig...",https://twitter.com/Maha_MEDD/status/127320617...,a map prepared by csir neeri mumbai showing co...,worry
3,"['#unlock1', '#mumbai']",https://twitter.com/theprasannata/status/12731...,i think probably us young people in the office...,neutral
4,"['#missionbeginagain', '#mumbai', '#unlock1']",https://twitter.com/mumbaiat24am/status/127318...,missionbeginagain mumbai crowds surge on day o...,happiness
...,...,...,...,...
353,"['#covid_19india', '#mumbai', '#socialdistanci...",https://twitter.com/MonilDalal/status/12699036...,government has started the economy but citizen...,joy
354,"['#coronapandemic', '#mumbai', '#mumbaikars', ...",https://twitter.com/NBhalwankar/status/1269896...,another shocker how can the coronapandemic get...,worry
355,"['#mumbai', '#monday', '#mondayblues', '#monda...",https://twitter.com/ompsyram/status/1269896580...,the new normal mumbai monday mondayblues monda...,neutral
356,"['#unlock1', '#covid19india']",https://twitter.com/ScribeUpma/status/12698952...,mumbai on work mode unlock covid india,worry


In [38]:
df0['sentiment'].value_counts()

worry        146
neutral      105
sadness       45
happiness     34
joy           28
Name: sentiment, dtype: int64

In [50]:
index = 96


print(df0['tweet'][index])
print(df0['sentiment'][index])

mumbai coronavirus updates missionbeginagain unlock suspected positive total positive patients deaths discharged today deaths in the city since includes old deaths
worry


In [46]:
df0.to_csv(r'C:\Users\Shibbs\Desktop\Projects\Sentiment analysis\My dataset\cnn_lstm1_7.3.csv')